# <div dir=rtl> תרגול שביעי - SVM, Validation, Cross-Validation </div>
<div dir=rtl>
    בתרגול זה נכיר כלי נוסף בעולם הלמידה המפוקחת ונלמד איך לבחון את עצמינו יותר טוב.
</div>

## <div dir=rtl>שאלה 1</div>

<div dir=rtl>
    בשנת 2008 פרסם הNIH האמריקאי מידע על אודות 4238 תושבי העיר פארמינגהם במדינת מסצ'וסטס. <br />
    המידע כלל נתונים על בריאותם של המשתתפים במחקר והאם בתוך עשור המשתתפים זוהו כסובלים ממחלות בכלי הדם הכליליים בלב. המידע זמין בקובץ 6_heart_disease_2.csv. <br />
</div>
<div dir=rtl style='background-color: #fcf2f2;'> 
    הערה: הדאטאסט אמיתי וניתן להורדה בצורה חופשייה. הנתונים בו שונו לצורך הדוגמה. <br />
</div><br />
<div dir=rtl>
    א. טענו את הקובץ לאובייקט מסוג DataFrame.
</div>

In [ ]:
%ls

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn import metrics


In [ ]:
df = pd.read_csv('6_heart_disease_2.csv')
df.sample(5)

<div dir=rtl>
    ב. בתרגול קודם הכרנו את כלל הפיצ'רים בדאטא סט והתרשמנו כי לחץ הדם הסיסטולי ולחץ הדם הדיאסטולי מהווים מדדים המפרידים במידה מסוימת בין המטופלים החולים והבריאים. <br />
    בתרגול הקודם התאמנו מודל לוגיסטי לכל הנתונים יחד וחישבנו את מדד הדיוק <br />
</div>
<br />

\begin{align}
Accuracy = \frac{TP+TN}{TP+TN+FP+FN}\\
\end{align}
<br />

<div dir=rtl>
    ראינו בתרגול הקודם שהפרדה גם בעזרת שני פיצ׳רים לא כל כך אינפורמטיבים (BMI ודופק) עדיין נתנו ציון דיוק גבוה, מעל 0.8. למה זה קורה?<br />
    ננסה להבין זאת ע״י דוגמה - נניח שיש לנו דאטה סט שמכיל 1000 תצפיות שמסומנות כ-0 ו10 תצפיות שמסומנת כ-1. <br />
    בנוסף נניח שיש לנו מודל קלאסיפיקציה ״מנוון״, כלומר לכל התצפיות המודל נותן את הפרדיקציה 0. מה יהיה ציון הדיוק?
</div>

In [ ]:
y_true = np.concatenate((np.zeros(1000), np.ones(10)))
y_pred = np.zeros(y_true.shape[0])

metrics.accuracy_score(y_true, y_pred)

<div dir=rtl>
    אבל המודל הזה ממש לא מודל טוב....אם נשים לב במודל הגרוע ביותר אנחנו נקבל שמדד הדיוק שווה ליחס בין כמות התצפיות של הקבוצה הגדולה ביותר לכל הדאטה סט - הTN תמיד יהיה מספר מאוד גדול! כי הדאטה ממש לא מאוזן <br />
    נתבונן בעוד שני מדדי דיוק - <code>precision</code> ו - <code>recall</code>
</div>
<br />

\begin{align}
Precision = \frac{TP}{TP+FP}\\
    Recall = \frac{TP}{TP+FN}\\
\end{align}
<br />
<div dir=rtl>
    מדד ה- <code>precision</code> מתאר כמה המודל מסווג טוב דוגמאות למחלקה החיובית (1) כלומר כמה תצפיות מתוך כל התצפיות שהכרזנו עליהן כחיוביות אכן חיוביות<br />
    מדד ה- <code>recall</code>, ידוע גם כ sensitivity מתאר כמה המודל מצליח לסווג נכונה את המחלקה החיובית מתוך סך כל ההופעות שלה בדאטה. 
</div>

In [ ]:
print(metrics.precision_score(y_true, y_pred))
print(metrics.recall_score(y_true, y_pred))

<div dir=rtl>
        בדקו האם הדאטה מאוזן או לא 
</div>

In [ ]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (10, 5),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large',
         'axes.grid':True,
         'axes.grid.which':'both'
         }
plt.rcParams.update(params)

In [ ]:
sns.countplot(x=df['TenYearCHD'], palette='coolwarm')
# shorter but the same as df['TenYearCHD'].value_counts().plot(kind='bar')


<div dir=rtl>
    ג. צרו חלוקה ברשומות כך שבקבוצה אחת יפלו 80% מהדוגמאות, ושאר הדוגמאות בקבוצה השניה. שימו לב שעליכם לחלק את הרשומות כך שהתפלגות המחלקות תישאר זהה בכל קבוצה.
</div>
<br />

<div dir=rtl>
    ראשית ננסה להבין מה התפלגות המחלקות בנתונים:
</div>

In [ ]:
pd.DataFrame(
    df['TenYearCHD'].value_counts(normalize=True)
)

<div dir=rtl>
    קבלנו שרוב הנתונים מגיעים מנבדקים בריאים - יש הטייה חזקה בדאטא לטובת המחלקה הזו. נשים לב שאם נבחר באקראי רשומות לכל קבוצה (אנחנו רוצים שתי קבוצות) החלוקה עלולה להיות כזו שמחלקה אחת תהיה מיוצגת בצורה לא טובה - או בייצוג חסר או בייצוג יתר. אנחנו רוצים לדאוג לכך שגם אחרי החלוקה לשתי קבוצות ההתפלגות תישמר 85-15.
    <br />
    אמנם ניתן לממש זאת בעצמינו, אבל למזלינו מימוש מוכן ובדוק כבר קיים:
</div>

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_val = train_test_split(
    df[['sysBP','diaBP', 'TenYearCHD']],
    train_size=0.8,
    stratify=df['TenYearCHD']
)
pd.DataFrame(x_train['TenYearCHD'].value_counts(normalize=True)),\
pd.DataFrame(x_val['TenYearCHD'].value_counts(normalize=True))

<div dir=rtl>
    נשים לב לפרמטר stratify (מאנגלית: סטרטיפיקציה - יצרת שכבות). הוא מנחה את הפונקציה לחלק את הנתונים לפי הלייבלים הנתונים לו. אכן בהסתכלות על ההתפלגות בכל קבוצה - מקבלים שהתפלגות הלייבלים נשמרה כמו במקור בדיוק גבוה.
</div>
<br />
<div dir=rtl>
    ד. צרו מודל לינארי לסיווג הנתונים לשתי מחלקות אשר ממקסם את המרחק בין שתי המחלקות והישר המפריד. יש ללמוד מהקבוצה הגדולה (זו שסומנה ב train)
</div>
<br />
<br />
<div dir=rtl>
    בדיוק לשם כך נוצר ה Support Vector Machine. <br />
    מדובר במודל אשר מתאים מפריד לינארי (קו ישר, מישור או מישור במימד גבוה) ואך ורק לינארי. בתהליך האימון המודל מתחשב במרחק בין הישר המפריד לדוגמאות הקרובות ביותר אליו מכל מחלקה (אלה ווקטורי התמך) ומעביר את הישר בניהם.

</div>

<div dir=rtl>
    את שתי המחלקות נסמן באופן הבא $y_i \in \{-1,1\}$ והמטרה שלנו היא למצוא ישר או מישור שמפריד בצורה הטובה ביותר את הנקודות של שתי המחלקות. <br />
    כלומר, נרצה שהפרדיקציות יהיו בעלות אותו הסימן :
</div>

\begin{align}
y_i(w^T \cdot x_i - b) ֿ\geq 1\\\forall i, 1 \leq i \leq n\\
\end{align}

<br /><br />
<div dir=rtl>
    הבעיה בדוגמה הזו היא שהדאטה לא תמיד יהיה ניתן להפרדה מושלמת ע״י קו לינארי. בשביל לאפשר יותר גמישות באלגוריתם נסתמך על soft-SVM שמאפשר סטייה מההפרדה הלינארית המושלמת ובעצם ״מעניש״ נקודות שמעבר לקו.<br />
    הפונקציה שנשתמש בה במקרה זה נקראת גם <code>hinge loss</code> והיא מקבלת את הערך 0 כאשר הנקודה נמצאת בצד הנכון של הישר, אחרת ערך הפרופורציונאלי למרחק של הנקודה מהישר
</div>


\begin{align}
l_i = \max(0, 1 - y_i(w^T \cdot x_i - b))\\
\end{align}


<div dir=rtl>
   בפועל נרצה למזער את הloss אבל לשמור על שוליים כמה שיותר גדולים
</div>

\begin{align}
\hat{w}, \hat{b} = ||w||^2 + C\sum{L_{hinge}(y_i, w^T \cdot x_i - b)}\\
\end{align}

<br /><br />
<div dir=rtl>
    הפרמטר C מגדיר את גודל הרגולריזציה והוא עוזר למודל להתכנס במצבים בהם יש "רעש" והדגימות לא מופרדות בצורה מושלמת על ידי ישר. הוא מאזן בין ערך ה- <code>loss</code> לבין גודל השוליים: <br />
    כאשר C גדול - עדיפות לשוליים קטנים יותר אם זה מוביל לקלאסיפיקציה טובה יותר.<br />
    עבור C קטן יותר - יותר משקל לשוליים רחבים ופחות לקלאסיפיקציה לא נכונה.
</div>


<div dir=rtl>
    צרו תרשים פיזור של שני הפיצ׳רים: sysBP, diaBP
</div>


In [ ]:
boxp = x_train[['sysBP','diaBP','TenYearCHD']].plot.scatter(
    x='diaBP',
    y='sysBP',
    c='TenYearCHD',
    cmap='jet',
    sharex=False
)

<div dir=rtl>
    כעת נתאים מודל SVM עבור קלסיפיקציה:
</div>

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='linear')
svc.fit(
    x_train[['diaBP','sysBP']], 
    x_train['TenYearCHD']
)


boxp = x_train[['sysBP','diaBP','TenYearCHD']].plot.scatter(
    x='diaBP',
    y='sysBP',
    c='TenYearCHD',
    cmap='jet',
    sharex=False
)

def decision_boundary_plot(results, min_x, max_x):
    # get the separating hyperplane
    w = results.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(min_x - 5, max_x + 5)  # make sure the line is long enough
    yy = a * xx - (results.intercept_[0]) / w[1]
    plt.plot(xx, yy, c='darkorange', lw=4)



decision_boundary_plot(svc, x_train['diaBP'].min(), x_train['diaBP'].max())

In [ ]:
# this is the same plot but now showing support vectors
plt.scatter(x_train['diaBP'], x_train['sysBP'], c=x_train['TenYearCHD'], s=30, cmap=plt.cm.Paired)

# plot the decision function
ax = plt.gca()
xlim = ax.get_xlim()
ylim = ax.get_ylim()

# create grid to evaluate model
xx = np.linspace(xlim[0], xlim[1], 30)
yy = np.linspace(ylim[0], ylim[1], 30)
YY, XX = np.meshgrid(yy, xx)
xy = np.vstack([XX.ravel(), YY.ravel()]).T
Z = svc.decision_function(xy).reshape(XX.shape)

# plot decision boundary and margins
ax.contour(
    XX, YY, Z, colors="k", levels=[-1, 0, 1], alpha=0.9, linestyles=["--", "-", "--"]
)

<div dir=rtl>
    ה. בחנו את ביצועי המודל הן על הקבוצת האימון והן על קבוצת הוולידציה.
</div>

In [ ]:
metrics.accuracy_score(
    x_train['TenYearCHD'],
    svc.predict(x_train[['diaBP','sysBP']])
)

In [ ]:
metrics.accuracy_score(
    x_val['TenYearCHD'],
    svc.predict(x_val[['diaBP','sysBP']])
)

<div dir=rtl>
    באופן צפוי - קיבלנו ביצועים פחות טובים על קבוצת הוולידציה מאשר על קבוצת האימון (אבל - זה לא חד משמעי). זה יכול לנבוע ממגוון סיבות - רעש בדוגמאות שבחרנו ללמידה, רעש בוולידציה, בחירה לא מוצלחת של נקודות ועוד.
    <br />
</div>
<br />
<div dir=rtl>
    ו. חיזרו על התהליך.
</div>

In [ ]:
x_train, x_val = train_test_split(
    df[['sysBP','diaBP', 'TenYearCHD']],
    train_size=0.8,
    stratify=df['TenYearCHD']
)

svc = SVC(kernel='linear')
svc.fit(
    x_train[['diaBP','sysBP']], 
    x_train['TenYearCHD']
)

metrics.accuracy_score(
    x_val['TenYearCHD'],
    svc.predict(x_val[['diaBP','sysBP']])
)

<div dir=rtl>
    למעשה אנחנו רואים שבחלוקות חוזרות ונשנות של הדאטא מקבלים נתונים שונים. נשאלות כעת השאלות: איך נבחר את הפרמטרים האופטימליים של המודל בצורה אמינה? על אילו נתונים נדווח?
</div>
<br />
<br />
<div dir=rtl>
    ז. חלקו את הנתונים לחמש חלוקות 80-20 ייחודיות וזרות אחת לשניה ואמנו מודל על כל חלוקה. ביחרו על בסיס זה את חוזק הרגולריזציה.
</div>
<br />
<div dir=rtl>
    כעת נכיר את מושג הקרוס-וולידציה. בשיטה זה משתמשים בכל הדאטא לאימון (למעט קבוצת מבחן בה לא נוגעים עד השלב האחרון). בכל סבב מאמנים מודל אחר ובודקים את התוצאות שלו.
    <br />
    קרוס וולידציה היא צורה אמינה יותר לדיווח על תוצאות משום שמבטלת את החלוקה כגורם המשפיע על המודל.
    <br />
    אמנם לא מסובך לממש זאת בעצמינו, אבל למזלינו קיים מימוש מוכן ובדוק.
</div>

![](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

![](7_cross_validation.png)

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

skf = StratifiedKFold(n_splits=5)

svc = SVC(kernel='linear')
scores = cross_val_score(
    svc,
    df[['sysBP', 'diaBP']],
    df['TenYearCHD'],
    cv=skf
)
scores

<div dir=rtl>
    נשים לב למעלה שקיבלנו 5 מספרים - חמישה ערכי דיוק. עוד נשים לב שהמודל בו אנחנו משתמשים, svc, הוא מודל שהחבילה "יודעת" לעבוד איתו, לכן התהליך כולו אוטומטי.
    <br />
    כעת, תוך שימוש בשיטה הזו נבחר פרמטר C מתאים:
</div>

In [ ]:
regularization_strength = [1e-5, 1e-4, 1e-3 ,0.01, 0.1, 1]

results = pd.DataFrame(
    [],
    columns=regularization_strength
)

for strength in regularization_strength:
    skf = StratifiedKFold(n_splits=5)
    svc = SVC(kernel='linear', C=strength)
    scores = cross_val_score(
        svc,
        df[['sysBP', 'diaBP']],
        df['TenYearCHD'],
        cv=skf
    )
    
    results.loc[:,strength] = scores
    
results

In [ ]:
sns.pointplot(
    data=results,
    ci='sd'
)

xl = plt.xlabel('Regularization Parameter')
yl = plt.ylabel('Accuracy')

<div dir=rtl>
    בתרשים למעלה קיבלנו את הממוצע וסטיית התקן של חמש ריצות שונות של המודל על חמש חלוקות שונות של הדאטא. רווח הסמך שבחרנו לבקש מהמודל להציג הוא הסטיית תקן. כעת נוכל לבחור פרמטר בצורה מושכלת ומוצדקת.
</div>